In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

In [2]:
class my_model(nn.Module):
    '''A simple model with 1 hidden layer. 7,10 and 3 neurons in input, hidden and output layer respectively.'''
    def __init__(self): 
        super(my_model, self).__init__()
        self.input_layer = nn.Linear(in_features=7, out_features=10)
        self.output_layer = nn.Linear(in_features=10, out_features=3)
    
    def forward(self, x1):
        output = self.output_layer(F.relu(self.input_layer(x1)))
        return output
    
model = my_model()  
summary(model, (1,7))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 10]              80
            Linear-2                 [-1, 1, 3]              33
Total params: 113
Trainable params: 113
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [3]:
import torch.optim as optim
from torch.optim import lr_scheduler

criterion = torch.nn.MSELoss(reduction=None)
optimizer = optim.SGD(model.parameters(), lr=2)

In [4]:
def get_lr(optimizer):
    '''Get Current Learning Rate'''
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [5]:
import copy, time
from torch.utils.tensorboard import SummaryWriter

def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    writer = SummaryWriter("./tensorboard_events")
    for epoch in range(num_epochs):
        print('-'*40)
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        optimizer.step()
        scheduler.step()
        lr = get_lr(optimizer)
        writer.add_scalar("Learning Rate", lr, epoch)
        print('Learning Rate', lr)
    writer.flush()
    writer.close()
    return model

In [6]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=1.0)

In [7]:
lrchanger = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1, last_epoch=- 1, verbose=True)
model_ft = train_model(model, criterion, optimizer, lrchanger, num_epochs=12)

Adjusting learning rate of group 0 to 1.0000e+00.


2022-08-10 17:43:58.232275: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-10 17:43:58.234624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 17:43:58.234631: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


----------------------------------------
Epoch 0/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning Rate 1.0
----------------------------------------
Epoch 1/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning Rate 1.0
----------------------------------------
Epoch 2/11
Adjusting learning rate of group 0 to 1.0000e-01.
Learning Rate 0.1
----------------------------------------
Epoch 3/11
Adjusting learning rate of group 0 to 1.0000e-01.
Learning Rate 0.1
----------------------------------------
Epoch 4/11
Adjusting learning rate of group 0 to 1.0000e-01.
Learning Rate 0.1
----------------------------------------
Epoch 5/11
Adjusting learning rate of group 0 to 1.0000e-02.
Learning Rate 0.010000000000000002
----------------------------------------
Epoch 6/11
Adjusting learning rate of group 0 to 1.0000e-02.
Learning Rate 0.010000000000000002
----------------------------------------
Epoch 7/11
Adjusting learning rate of group 0 to 1.0000e-02.
Learning Rate 0.01000000

In [8]:
def func1(epoch):
    return 1 if epoch%2==0 else 0.5

lrchanger = lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=func1, verbose=True)
model_ft = train_model(model, criterion, optimizer, lrchanger, num_epochs=10)
'''Takes function as input which inturn takes epoch number as input. The function
returns a  number which is multiplied with current lr to give new lr'''

Adjusting learning rate of group 0 to 1.0000e-04.
----------------------------------------
Epoch 0/9
Adjusting learning rate of group 0 to 5.0000e-05.
Learning Rate 5.0000000000000016e-05
----------------------------------------
Epoch 1/9
Adjusting learning rate of group 0 to 5.0000e-05.
Learning Rate 5.0000000000000016e-05
----------------------------------------
Epoch 2/9
Adjusting learning rate of group 0 to 2.5000e-05.
Learning Rate 2.5000000000000008e-05
----------------------------------------
Epoch 3/9
Adjusting learning rate of group 0 to 2.5000e-05.
Learning Rate 2.5000000000000008e-05
----------------------------------------
Epoch 4/9
Adjusting learning rate of group 0 to 1.2500e-05.
Learning Rate 1.2500000000000004e-05
----------------------------------------
Epoch 5/9
Adjusting learning rate of group 0 to 1.2500e-05.
Learning Rate 1.2500000000000004e-05
----------------------------------------
Epoch 6/9
Adjusting learning rate of group 0 to 6.2500e-06.
Learning Rate 6.25000

'Takes function as input which inturn takes epoch number as input. The function\nreturns a  number which is multiplied with current lr to give new lr'

In [9]:
lambda1 = lambda epoch:1
lrchanger = lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1], verbose=True)
model_ft = train_model(model, criterion, optimizer, lrchanger, num_epochs=12)
'''useful if optimizer has multiple groups and each group needs a unique custom
function to calculate the learning rate. check official documentation'''

Adjusting learning rate of group 0 to 1.0000e+00.
----------------------------------------
Epoch 0/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning Rate 1.0
----------------------------------------
Epoch 1/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning Rate 1.0
----------------------------------------
Epoch 2/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning Rate 1.0
----------------------------------------
Epoch 3/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning Rate 1.0
----------------------------------------
Epoch 4/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning Rate 1.0
----------------------------------------
Epoch 5/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning Rate 1.0
----------------------------------------
Epoch 6/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning Rate 1.0
----------------------------------------
Epoch 7/11
Adjusting learning rate of group 0 to 1.0000e+00.
Learning

'useful if optimizer has multiple groups and each group needs a unique custom\nfunction to calculate the learning rate. check official documentation'

In [17]:
optimizer = optim.SGD(model.parameters(), lr=0.8)
print('Initial lr set at optimizer = 0.8')
lrchanger=lr_scheduler.ConstantLR(optimizer, factor=0.05, total_iters=7, last_epoch=-1, verbose=True)
model_ft = train_model(model, criterion, optimizer, lrchanger, num_epochs=12)

'''Decays lr by a constant value = 'factor' for ('total_iters' -1 ) number of epochs'''

Initial lr = 0.8
Adjusting learning rate of group 0 to 4.0000e-02.
----------------------------------------
Epoch 0/11
Adjusting learning rate of group 0 to 4.0000e-02.
Learning Rate 0.04000000000000001
----------------------------------------
Epoch 1/11
Adjusting learning rate of group 0 to 4.0000e-02.
Learning Rate 0.04000000000000001
----------------------------------------
Epoch 2/11
Adjusting learning rate of group 0 to 4.0000e-02.
Learning Rate 0.04000000000000001
----------------------------------------
Epoch 3/11
Adjusting learning rate of group 0 to 4.0000e-02.
Learning Rate 0.04000000000000001
----------------------------------------
Epoch 4/11
Adjusting learning rate of group 0 to 4.0000e-02.
Learning Rate 0.04000000000000001
----------------------------------------
Epoch 5/11
Adjusting learning rate of group 0 to 4.0000e-02.
Learning Rate 0.04000000000000001
----------------------------------------
Epoch 6/11
Adjusting learning rate of group 0 to 8.0000e-01.
Learning Rate 0

"Decays lr by a constant value = 'factor' for ('total_iters' -1 ) number of epochs"